In [1]:
import pandas as pd

## Data Preparation

In [22]:
#load data
df_seq=pd.read_csv('pdb_data_seq.csv')
df_properties=pd.read_csv('pdb_data_no_dups.csv')
df_total=df_seq.merge(df_properties,left_on='structureId',right_on = 'structureId')


In [23]:
df_total.columns

Index(['structureId', 'chainId', 'sequence', 'residueCount_x',
       'macromoleculeType_x', 'classification', 'experimentalTechnique',
       'macromoleculeType_y', 'residueCount_y', 'resolution',
       'structureMolecularWeight', 'crystallizationMethod',
       'crystallizationTempK', 'densityMatthews', 'densityPercentSol',
       'pdbxDetails', 'phValue', 'publicationYear'],
      dtype='object')

#### Select only protein, and filtered by top N

In [24]:
#select the data in top n by count, top 10?
count = df_total['classification'].value_counts(dropna=False)[:10]
df_selected=df_total[df_total['classification'].isin(set(count.index))]
#we want only protein
df_selected=df_selected[df_selected['macromoleculeType_x'].isin(set(['Protein']))]

#### Select proteins with only one chain

In [25]:
#select proteins with only one chain in the data set
#how to justify this operation?
df_onechain = df_selected[df_selected.groupby('structureId').structureId.transform(len) == 1]

In [26]:
#test_df = df_selected[['structureId','classification','sequence']]
test_df = df_onechain[['structureId','classification','sequence']]

Things to be done

Further select data and simplify problem, select proteins with only one chain?


Figure out how to convert sequence data into array and training model afterwards.


More models and discussion (*LSTM)

Models build on features other than sequence.



## Model Training

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier,ExtraTreesClassifier

In [28]:
#optional, take part of the data for faster verification
data = test_df#.sample(50000)

#need to remove nulls
data = data.dropna()
X_train, X_test,y_train,y_test = \
train_test_split(data['sequence'], data['classification'], test_size = 0.2, random_state = 1)

#### Feature Extraction From Sequence Data

In [29]:
#vectorize data, prepare for building models
#Convert a collection of text documents to a matrix of token counts
#seems has nothing to do with sequence but only with the frequency

#ngram is a parameter we need to focus on, 

#vect = CountVectorizer(analyzer = 'char_wb', ngram_range = (3,3))
vect = CountVectorizer(analyzer = 'char_wb')

# Fit and Transform CountVectorizer
#occasionally may meet np.nan error
vect.fit(X_train)
X_train_df = vect.transform(X_train)
X_test_df = vect.transform(X_test)

#to store the results for different mothods
prediction = dict()

#### Naive Bayes

In [30]:
model = MultinomialNB()
model.fit(X_train_df, y_train)
#test on test set
NB_pred = model.predict(X_test_df)
prediction["MultinomialNB"] = accuracy_score(NB_pred, y_test)
print( prediction['MultinomialNB'])

0.42338555598012995


#### adaboost

In [31]:
model = AdaBoostClassifier()
model.fit(X_train_df,y_train)
ADA_pred = model.predict(X_test_df)
prediction["Adaboost"] = accuracy_score(ADA_pred , y_test)
print(prediction["Adaboost"])

0.41708062667176155


#### RandomForestClassifier

In [32]:
model = RandomForestClassifier()
model.fit(X_train_df,y_train)
ADA_pred = model.predict(X_test_df)
prediction["Random_Forest"] = accuracy_score(ADA_pred , y_test)
print(prediction["Random_Forest"])

D:\Users\ericl\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8372181887657624
